In [30]:
import librosa 
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import os 
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [31]:
# refer to paper for the features to be extracted
def extract_features(signal):

    result = []
    features = []

    spectral_centroid = librosa.feature.spectral_centroid(y=signal)
    spectral_rollof = librosa.feature.spectral_rolloff(y=signal)
    spectral_flux = librosa.onset.onset_strength(y=signal,center=True)
    zero_crossing = librosa.feature.zero_crossing_rate(y=signal)
    rms = librosa.feature.rms(y=signal)
    mfcc = librosa.feature.mfcc(y=signal)

    features.append(spectral_centroid)
    features.append(spectral_rollof)
    features.append(spectral_flux)
    features.append(zero_crossing)
    features.append(rms)

    for feature in features:
        result.append(np.mean(feature))
        result.append(np.std(feature))

    for i in range(0, 5):
        result.append(np.mean(mfcc[i, :]))
        result.append(np.std(mfcc[i, :]))

    return result

In [32]:
genre_directory_path = os.path.join(os.getcwd(),"genres")
labels = os.listdir(genre_directory_path)
dataset = None
target = []
for label in labels:
    data_path = os.path.join(genre_directory_path,label)
    for file in os.listdir(data_path):
        audio_path = os.path.join(data_path,file)
        audio_sample,sr = librosa.load(audio_path)
        audio_sample = np.array(audio_sample)
        feature = extract_features(audio_sample)
        if dataset is None:
            dataset = np.array(feature)
        else:
            dataset = np.vstack((dataset,feature))
        target.append(label)

In [33]:
X_train,X_test,y_train,y_test = train_test_split(dataset,target,test_size=0.20,random_state=42)
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train = scaler.fit_transform(X_train)
knn = KNeighborsClassifier(3)
knn.fit(X_train,y_train)
X_test = scaler.transform(X_test)
y_pred = knn.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.615
